In [ ]:
#Step 1- Download this repository and upload to "/content/gdrive/My Drive" that is in your google drive.
#After that your folders will seen like below.
#Note: Do not forget to change Runtime→Change Runtime Type and select Hardware accelerator as GPU.
"""
TensorFlow
├───scripts
│   └───preprocessing
│     └───generate_tfrecord.py 
└───workspace
    └───training_demo
        ├───annotations
        │   └───label_map.pbtxt 
        ├───content
        │     └───sheep_videos.mp4 
        ├───exported-models
        ├───images
        │   ├───test
        │   │     └───test images with corresponding XML files
        │   └───train
        │         └───train images with corresponding XML files
        ├───models
        │   └───my_ssd_resnet50_v1_fpn
        └───pre-trained-models
            └───ssd_resnet50_v1_fpn_640x640_coco17_tpu-8
  """

In [ ]:
#Step 2- Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Step 3- Download TensorFlow Model Garden
%cd /content/gdrive/My Drive/TensorFlow
!git clone https://github.com/tensorflow/models
#You have a new folder named ‘models’ in your 'TensorFlow/' directory!

In [ ]:
#Step 4- Download Pre-trained Model
%cd /content/gdrive/My Drive/TensorFlow/workspace/training_demo/pre-trained-models
import os
import shutil
import glob
import urllib
import tarfile
from requests import get

MODEL = 'ssd_resnet50_v1_fpn_640x640_coco17_tpu-8'
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'

if not (os.path.exists(MODEL_FILE)):
  with open(MODEL_FILE, "wb") as file:
    # get request
    response = get(DOWNLOAD_BASE + MODEL_FILE)
    # write to file
    file.write(response.content)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()
os.remove(MODEL_FILE)
#You have a pretrained model in your 'TensorFlow/workspace/training_demo/pre-trained-models' directory!
#Move the 'pipeline.config' file from ‘TensorFlow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8’ to "TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn"
shutil.move("ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config", "../models/my_ssd_resnet50_v1_fpn") 


/content/gdrive/My Drive/TensorFlow/workspace/training_demo/pre-trained-models


'../models/pipeline.config'

In [ ]:
#Step 5- Configure the pipeline file.
#Go to "TensorFlow/workspace/training_demo/models/my_ssd_resnet50_v1_fpn".
#Open the pipeline.config file.
#Change the lines shown below
"""
Line 3:
  num_classes: 1 

Line 131:
  batch_size: 8

Line 161:
  fine_tune_checkpoint: "pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0" (#path to checkpoint of downloaded pre-trained-model)

Line 162:
  num_steps: 25000 (#maximum number of steps to train model, you can stop model training on any step you wish)

Line 167:
  fine_tune_checkpoint_type: "detection"

Line 168:
  use_bfloat16: false (#Set this to true only if you are training on a TPU)

Line 172:
  label_map_path: "annotations/label_map.pbtxt" (#path to your label_map file)

Line 174:
  input_path: "annotations/train.record" (#path to train.record)

Line 182:
  label_map_path: "annotations/label_map.pbtxt" (#path to your label_map file)

Line 186:
  input_path: "annotations/test.record" (#Path to test.record)
"""

In [ ]:
 #Step 6- Install some required libraries and tools
!apt-get install protobuf-compiler python-lxml python-pil
!pip install Cython pandas tf-slim

In [ ]:
#Step 7- Compile the Protobuf libraries
%cd '/content/gdrive/My Drive/TensorFlow/models/research/'
!protoc object_detection/protos/*.proto --python_out=.

/content/gdrive/My Drive/TensorFlow/models/research


In [ ]:
#Step 8- Set the environment to Build and Install setup.py.
import os
import sys
os.environ['PYTHONPATH']+=":/content/gdrive/My Drive/TensorFlow/models"
sys.path.append("/content/gdrive/My Drive/TensorFlow/models/research")

In [ ]:
#Step 9- Build and Install setup.py.
!python setup.py build
!python setup.py install

In [ ]:
#Step 10- Test the installation.
%cd '/content/gdrive/My Drive/TensorFlow/models/research/object_detection/builders/'
!python model_builder_tf2_test.py
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
print('Done')

In [ ]:
#Step 11- Generate Tfrecords
#cd into preprocessing directory
%cd '/content/gdrive/My Drive/TensorFlow/scripts/preprocessing'
#run the cell to generate test.record and train.record
!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/images/train' -l '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/train.record'
!python generate_tfrecord.py -x '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/images/test' -l '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt' -o '/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/test.record'
#You have two new files “test.record” and “train.record” in ‘TensorFlow/workspace/training_demo/workspace/training_demo/annotations’ folder.

/content/gdrive/My Drive/TensorFlow/scripts/preprocessing
Successfully created the TFRecord file: /content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/train.record
Successfully created the TFRecord file: /content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/test.record


In [ ]:
#Step 12- Start TensorBoard
#cd into training_demo
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'
#start the Tensorboard
%load_ext tensorboard
%tensorboard --logdir=models/my_ssd_resnet50_v1_fpn

In [ ]:
#Step 13- Train the Model
pip install lvis
!python model_main_tf2.py --model_dir=models/my_ssd_resnet50_v1_fpn --pipeline_config_path=models/my_ssd_resnet50_v1_fpn/pipeline.config

2021-01-08 06:40:48.484254: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-08 06:40:51.781805: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-08 06:40:51.782768: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-08 06:40:51.814781: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-01-08 06:40:51.815368: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2021-01-08 06:40:51.815403: I tensorflow/stream_executor/platform/default/dso_loade

In [ ]:
#Step 14- Export the Trained Model
!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path ./models/my_ssd_resnet50_v1_fpn/pipeline.config --trained_checkpoint_dir ./models/my_ssd_resnet50_v1_fpn/ --output_directory ./exported-models/my_model
#You have fine-tuned pretrained model in TensorFlow/workspace/training_demo/exported-models/my_model

In [ ]:
#Step 15- Loading the saved_model
import tensorflow as tf
import time
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
PATH_TO_SAVED_MODEL="/content/gdrive/My Drive/TensorFlow/workspace/training_demo/exported-models/my_model/saved_model"
print('Loading model...', end='')
# Load saved model and build the detection function
detect_fn=tf.saved_model.load(PATH_TO_SAVED_MODEL)
print('Done!')

In [ ]:
#Step 16- Loading the label_map
category_index=label_map_util.create_category_index_from_labelmap("/content/gdrive/My Drive/TensorFlow/workspace/training_demo/annotations/label_map.pbtxt",use_display_name=True)

In [ ]:
#Step 17- Define 'show_inference' function to detection per by per image .
%cd '/content/gdrive/My Drive/TensorFlow/workspace/training_demo'
import numpy as np
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def show_inference(frame):

    image_np = np.array(frame)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor=tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor=input_tensor[tf.newaxis, ...]

    detections=detect_fn(input_tensor)
    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections=int(detections.pop('num_detections'))
    detections={key:value[0,:num_detections].numpy()
                   for key,value in detections.items()}
    detections['num_detections']=num_detections

    # detection_classes should be ints.
    detections['detection_classes']=detections['detection_classes'].astype(np.int64)

    image_np_with_detections=image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=100,     #max number of bounding boxes in the image
          min_score_thresh=.3,      #min prediction threshold
          agnostic_mode=False)
    return image_np_with_detections


In [ ]:
#Step 18- Running the Inference.
#Read video in 'TensorFlow/workspace/training_demo/content/sheep_videos.mp4'
#Detect sheeps and save video that has detected sheep that drawn on rectangle in 'TensorFlow/workspace/training_demo/content/output.avi'
import cv2

video_capture = cv2.VideoCapture("content/sheep_videos.mp4")
frame_width = int(video_capture.get(3))
frame_height = int(video_capture.get(4))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter('content/output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

while video_capture.isOpened():
    # Capture frame-by-frame
    re,frame = video_capture.read()
    if re==True:
      Imagenp=show_inference(frame)
      out.write(Imagenp)
      if cv2.waitKey(25) & 0xFF == ord('q'):
          break
    else:
      break
video_capture.release()
out.release()
cv2.destroyAllWindows()

